**Clustering K-Anonymity with lower Information Loss**

#################**IMPORT LIBRARIES**##################

In [30]:
import pandas as pd
import math
import numpy as np
from tqdm import tqdm
import re
import random
from IPython.display import clear_output
from anytree import Node, RenderTree, PreOrderIter
from anytree.util import commonancestors

################**IMPORT DATA TABLE**#####################
Chọn 1 trong 3 Data Table để chạy 

**Adult Data Table**

In [31]:
col = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
dfin = pd.read_csv('adult.data' ,names=col, delimiter=',\s+', encoding='utf-8', engine='python')
print(dfin.shape)
for a in ['workclass', 'education', 'marital-status', 'occupation', 'native-country']:
    dfin = dfin[dfin[a] != '?']
dfin.replace({'Outlying-US(Guam-USVI-etc)':'Outlying-US[Guam-USVI-etc]'}, inplace=True)
#dfin = dfin.sample(500)
#dfin = dfin.iloc[0:500]
dfin.shape

(32561, 15)


(30162, 15)

**Phuong's Data Table**

In [3]:
dfin = pd.read_csv('input.csv', index_col=0)
dfin = dfin.sample(20)
dfin

,wage_month,region,head,sex,age,married,schooling,position,economictype,ky hop dong lao dong?,...,urban,Kinh,household_size,poor,houses,pay_rent,other_land/house,wealthindex,t_quantity durables,t_value durables
4708,6000,region3,1,male,36,0,5,position4,economictype4,2,...,0,0,2,0,1,0,0,95,10,18370
3408,3500,region6,1,male,43,1,3,position6,economictype4,2,...,0,1,4,0,1,0,0,155,19,11000
2423,5000,region1,0,female,28,1,14,position2,economictype2,2,...,0,1,5,1,1,0,0,155,13,28950
961,2800,region2,0,female,32,1,9,position6,economictype3,1,...,0,0,6,0,1,0,0,135,26,23150
3138,7000,region6,0,female,32,1,16,position3,economictype1,1,...,1,1,6,0,1,0,0,145,15,26600
3041,4200,region6,0,male,27,1,16,position2,economictype2,1,...,1,1,6,0,1,0,0,180,36,111500
3503,5895,region6,1,female,51,1,16,position2,economictype2,1,...,1,1,3,0,1,0,0,180,44,156680
3416,4100,region6,1,male,37,1,16,position2,economictype2,1,...,1,1,4,0,1,0,0,180,24,57120
643,10400,region2,0,female,53,1,16,position1,economictype2,2,...,1,2,4,0,1,0,0,150,13,71800
5154,4000,region4,0,female,33,1,15,position2,economictype2,1,...,0,1,4,0,1,0,0,125,19,40550


#####################**USER INPUT**############################

In [35]:
#Choose Quasi-identifiers

#LIST_QI = ['sex','age','region','married','position','economictype'] #Phuong data
LIST_QI = ['age', 'workclass', 'education', 'marital-status', 'occupation', 'race', 'sex', 'native-country'] #Adult data
#LIST_QI = ['Race', 'BirthDate', 'Gender', 'ZIP'] #Example data


#Choose hierarchy tree QIs
LIST_QI_HIERARCHY = ['workclass', 'education', 'marital-status', 'occupation', 'native-country']
#LIST_QI_HIERARCHY = ['region']

#CONVERT CATEGORICAL QIs to NUMERIC? DEFAULT=False
NUM_CATEGORICAL = False

In [4]:
dfin[['sex','age','region','married','position','economictype','wage_month']]

KeyError: "['region', 'married', 'wage_month', 'economictype', 'position'] not in index"

In [36]:
#Please input hierarchy tree for each hierarchy QIs:

#Hướng dẫn:
#Các node ngang hàng: '/'. Node cha của các node con1, con2: 'cha(con1/con2)'.
#Vui lòng input ít nhất 2 node con cho mỗi node cha.

#TREE STRING EXAMPLE: A(B/C) với A là root B, C là node con của A
#-------------------- A(B(D/E)/C(G/H)) A là root; D, E là node con của B
#-------------------- G, H là node con của C; B, C là node con của A 

region = 'Vietnam(mienBac(region1/region2)/mienNam(region4/region5)/mienTrung(region3/region6)'

workclass = 'Any(Worked(With-Pay(Private/Self-emp(Self-emp-not-inc/Self-emp-inc)/Gov(Federal-gov/Local-gov/State-gov))/Without-pay)/Never-worked)'
education = 'Any(Without-Post-Secondary(Preschool/Elementary(1st-4th/5th-6th/7th-8th)/Secondary(Junior-Secondary(9th/10th)/Senior-Secondary(11th/12th/HS-grad))))/Post-secondary(Some-college/Assoc(Assoc-acdm/Assoc-voc)/University(Bachelors/Prof-school)/Post-grad(Masters/Doctorate))'
martial_status = 'Any(Married(Married-civ-spouse/Married-AF-spouse)/Leave(Divorced/Separated)/Alone(Widowed/Married-spouse-absent)/Never-married)'
occupation = 'Any(White-collar(Exec-managerial/Prof-specialty/Sales/Adm-clerical)/Blue-collar(Tech-support/Craft-repair/Machine-op-inspct/Handlers-cleaners/Transport-moving/Priv-house-serv)/Other(Protective-serv/Armed-Forces/Farming-fishing/Other-service))'
native_country = 'Any(North-America(Canada/USA(United-States/Outlying-US[Guam-USVI-etc]))/Middle-America(Mexico/Honduras/Guatemala/Nicaragua/El-Salvador)/South-America(Columbia/Western-South-America(Ecuador/Peru)/Caribbean(Puerto-Rico/Dominican-Republic/Jamaica/Cuba/Haiti/Trinadad&Tobago))/Europe(Portugal/Yugoslavia/Western-Europe(France/England/Ireland/Scotland/Holand-Netherlands)/Southern-Europe(Italy/Greece)/Central-Europe(Hungary/Germany/Poland))/Asia(India/Iran/Southeastern-Asia(Philippines/Thailand/Cambodia/Vietnam/Laos)/Eastern-Asia(Japan/China/Hong/Taiwan/South))'

tempdict = [workclass, education, martial_status, occupation, native_country]
#tempdict = [region]
counterr = 0
hierarchyDict = {}
for qi in LIST_QI_HIERARCHY:
    #hierarchyDict.update({qi: input('Vui lòng điền cây phân cấp cho cột %s:' %(qi))})
    hierarchyDict.update({qi: tempdict[counterr]})
    counterr += 1

In [37]:
#Choose k (min:2) :
k = 500

############################*END OF USER INPUT*################################

#####################**FUNCTIONS** AND **CLASSES** DEFINITIONS##################

In [38]:
#Hàm phân loại quasi-identifiers theo numeric or categorical or hierarchy
#Tạo dictionary QI để dùng cho các hàm khác
qiClass = {}
def classifyNumCtg(df, qi):
    #Convert numeric to it's type
    for i in qi:
        df[i] = pd.to_numeric(df[i], errors='ignore')
    #Classify QI Numeric and QI Categorical
    qiNumeric = []
    qiCategorical = []
    for i in qi:
        if df[i].dtype == 'O':
            qiCategorical.append(i)
        else:
            qiNumeric.append(i)
    return {'numeric':qiNumeric, 'categorical':qiCategorical, 'hierarchy':{}}

In [39]:
#Hàm chuyển đổi input từ user sang dạng cây phân cấp:
#TREE STRING EXAMPLE: A(B/C) với A là root B, C là node con của A
#-------------------- A(B(D/E)/C(G/H)) A là root; D, E là node con của B
#-------------------- G, H là node con của C; B, C là node con của A 
def toTree(tree_string):
    x = re.split('[()]', tree_string)
    x = [i for i in x if i]
    rootNode = 0
    parentNode = []
    leafNode = []
    counterParent = 0
    counterLeaf = 0
    for item in x:
        if len(re.findall('/', item)) == 0:
            flag = 0
            if rootNode == 0:
                rootNode = Node(item)
            else:
                if counterParent == 0:
                    parentNode.append(Node(item, parent=rootNode))
                else:
                    parentNode.append(Node(item, parent=parentNode[counterParent-1]))
                counterParent = counterParent + 1
        else:
            if not re.search('^/', item):
                if flag != 1:
                    for i in re.split('/', item):
                        try:
                            leafNode.append(Node(i, parent=parentNode[counterParent-1]))
                        except:
                            leafNode.append(Node(i, parent=rootNode))
                        counterLeaf = counterLeaf + 1
                if flag == 1:
                    counterLeaf = counterLeaf - 1
                    parentNode.append(leafNode[counterLeaf])
                    leafNode.remove(leafNode[counterLeaf])
                    counterParent = counterParent + 1
                    for i in re.split('/', item):
                        leafNode.append(Node(i, parent=parentNode[counterParent-1]))
                        counterLeaf = counterLeaf + 1
                flag = 1
            else:
                if len(re.findall('/', item)) < 2:
                    if flag != 2:
                        if item == x[-1]:
                            if str('))' + item) in tree_string:
                                leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent))
                            else:
                                leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent))
                            counterLeaf = counterLeaf + 1
                        else:
                            if str('))))' + item) in tree_string:
                                parentNode.append(Node(item.split('/')[1], parent=rootNode))
                            elif str('))' + item) in tree_string:
                                parentNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent))
                            else:
                                parentNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent))
                            counterParent = counterParent + 1
                    if flag == 2:
                        counterParent = counterParent - 1
                        leafNode.append(parentNode[counterParent])
                        parentNode.remove(parentNode[counterParent])
                        counterLeaf = counterLeaf + 1
                        if str(')' + item) in tree_string:
                            leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent.parent))
                        else:
                            leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent))
                        counterLeaf = counterLeaf + 1
                    flag = 2
                else:
                    pass
    return [rootNode, parentNode, leafNode]

In [40]:
#Hàm chuyển QI kiểu categorical về dạng numeric:
def convertQIValue(df, qilist):
    fullMapDict = {}
    for qi in qilist:
        oldValue = df.sort_values(by=qi)[qi].unique()
        newValue = []
        for i in range(len(df[qi].unique())):
            if len(df[qi].unique()) == 2:
                #newValue.append(qi + str(i))
                newValue.append(i)
            else:
                newValue.append(i+1)
        mapDict = dict(zip(oldValue, newValue))
        df[qi] = df[qi].replace(mapDict)
        fullMapDict.update({qi: mapDict})
    return fullMapDict

In [41]:
## Hàm tính khoảng cách giữa 2 records (rows)
def distance(r1, r2, df, qiClass): #(row1, row2, dataframe, quasi-identifers class)
    totalDistance = 0
    numDistance = 0
    catgDistance = 0
    hieDistance = 0
    for qi in qiClass['numeric']:
        if df[qi].max() == df[qi].min():
            numDistance += 0
        else:
            numDistance += abs(r1[qi]-r2[qi])/(df[qi].max() - df[qi].min())
    for qi in qiClass['categorical']:
        if NUM_CATEGORICAL:
            if df[qi].max() == df[qi].min():
                catgDistance += 0
            else:
                catgDistance += abs(r1[qi]-r2[qi])/(df[qi].max() - df[qi].min())
        else:
            if r1[qi] != r2[qi]:
                catgDistance += 1
    for qi in qiClass['hierarchy'].keys():
        if r1[qi] != r2[qi]:
            r1_ac = set(r1[qi].ancestors)
            r2_ac = set(r2[qi].ancestors)
            try:
                comanc = list(r1_ac.intersection(r2_ac))[-1]
            except:
                comanc = qiClass['hierarchy'][qi]['root']
            #FIX THIS SECTION
            #Lỗi do hàm commonancestor với centroid 
            numLeavesCA = len(list(PreOrderIter(comanc, filter_=lambda node: node.is_leaf)))
            totalLeaves = len(qiClass['hierarchy'][qi]['children'])
            hieDistance += numLeavesCA/totalLeaves
    totalDistance = numDistance + catgDistance + hieDistance
    return totalDistance

In [42]:
## Định nghĩa lớp phân cụm:
class Cluster:
    def __init__(self, centroid):
        self.centroid = centroid
        self.records = pd.DataFrame(columns=self.centroid.index)
        self.records = self.records.append(self.centroid)
        self.informationLoss = 0
    def updateCentroid(self):
        for qi in qiClass['numeric']:
            self.centroid[qi] = self.records[qi].mean()
        for qi in qiClass['categorical']:
            if NUM_CATEGORICAL:
                self.centroid[qi] = self.records[qi].mean()
            else:
                if len(self.records[qi].unique()) > 1:
                    self.centroid[qi] = qi
                else:
                    self.centroid[qi] = self.records[qi].unique()[0]
        for qi in qiClass['hierarchy']:
            listAncestor = []
            for item in self.records[qi].unique():
                listAncestor.append(set(item.ancestors))
            for n in range(len(listAncestor)):
                commonA = listAncestor[0].intersection(listAncestor[n])
            commonA = list(commonA)
            lowestCommonA = commonA[0]
            for i in commonA:
                if len(i.path) > len(lowestCommonA.path):
                    lowestCommonA = i
            self.centroid[qi] = lowestCommonA
    def updateIL(self):
        PIL = 0
        numDistance = 0
        catgDistance = 0
        hieDistance = 0
        for qi in qiClass['numeric']:
            if dfsorted[qi].max() == dfsorted[qi].min():
                numDistance += 0
            else:
                numDistance += abs(self.records[qi].max() - self.records[qi].min())/(dfsorted[qi].max() - dfsorted[qi].min())
        for qi in qiClass['categorical']:
            if NUM_CATEGORICAL:
                if dfsorted[qi].max() == dfsorted[qi].min():
                    catgDistance += 0
                else:
                    catgDistance += abs(self.records[qi].max() - self.records[qi].min())/(dfsorted[qi].max() - dfsorted[qi].min())
            else:
                if len(self.records[qi].unique()) > 1:
                    catgDistance += 1
        for qi in qiClass['hierarchy'].keys():
            listAncestor = []
            for item in self.records[qi].unique():
                listAncestor.append(set(item.ancestors))
            for n in range(len(listAncestor)):
                commonA = listAncestor[0].intersection(listAncestor[n])
            lowestCommonA = list(commonA)[-1]

            numLeavesCA = len(list(PreOrderIter(lowestCommonA, filter_=lambda node: node.is_leaf)))
            totalLeaves = len(qiClass['hierarchy'][qi]['children'])
            hieDistance += numLeavesCA/totalLeaves
        PIL = numDistance + catgDistance + hieDistance
        self.informationLoss = PIL

In [43]:
#Convert string type to tree node type:
def toNode(df):
    for qi in qiClass['hierarchy']:
        leavesDict = {}
        for leaf in qiClass['hierarchy'][qi]['children']:
            leavesDict.update({leaf.name:leaf})
        df[qi] = df[qi].replace(leavesDict)
    return df   

#####################*END OF FUNCTIONS AND CLASSES DEFINITIONS*######################

In [44]:
#Phân loại Quasi-identifiers
qiClass = classifyNumCtg(dfin, LIST_QI)
qiClass

{'numeric': ['age'],
 'categorical': ['workclass',
  'education',
  'marital-status',
  'occupation',
  'race',
  'sex',
  'native-country'],
 'hierarchy': {}}

In [45]:
#Lấy chuỗi phân cấp cho các QI cây phân cấp
for qi in LIST_QI_HIERARCHY:
    if qi in qiClass['categorical']:
        qiClass['categorical'].remove(qi)
        qiClass['hierarchy'].update(hierarchyDict)
    else:
        print('Error %s can not be hierarchy' %(qi))
display(qiClass)

{'numeric': ['age'],
 'categorical': ['race', 'sex'],
 'hierarchy': {'workclass': 'Any(Worked(With-Pay(Private/Self-emp(Self-emp-not-inc/Self-emp-inc)/Gov(Federal-gov/Local-gov/State-gov))/Without-pay)/Never-worked)',
  'education': 'Any(Without-Post-Secondary(Preschool/Elementary(1st-4th/5th-6th/7th-8th)/Secondary(Junior-Secondary(9th/10th)/Senior-Secondary(11th/12th/HS-grad))))/Post-secondary(Some-college/Assoc(Assoc-acdm/Assoc-voc)/University(Bachelors/Prof-school)/Post-grad(Masters/Doctorate))',
  'marital-status': 'Any(Married(Married-civ-spouse/Married-AF-spouse)/Leave(Divorced/Separated)/Alone(Widowed/Married-spouse-absent)/Never-married)',
  'occupation': 'Any(White-collar(Exec-managerial/Prof-specialty/Sales/Adm-clerical)/Blue-collar(Tech-support/Craft-repair/Machine-op-inspct/Handlers-cleaners/Transport-moving/Priv-house-serv)/Other(Protective-serv/Armed-Forces/Farming-fishing/Other-service))',
  'native-country': 'Any(North-America(Canada/USA(United-States/Outlying-US[Guam-U

In [46]:
#Chuyển chuỗi thành cây phân cấp và gán lại vào biến qiClass
#Vẽ cây phân cấp của từng QIs 
for qi in qiClass['hierarchy']:
    qiClass['hierarchy'][qi] = {'root':toTree(qiClass['hierarchy'][qi])[0],
                                'parent':toTree(qiClass['hierarchy'][qi])[1],
                                'children':toTree(qiClass['hierarchy'][qi])[2]}

In [47]:
#Chuyển các categorical QIs về dạng số nếu NUM_CATEGORICAL = True
if NUM_CATEGORICAL:
    conversion = convertQIValue(dfin, qiClass['categorical'])
else:
    conversion = {}
conversion

{}

In [48]:
#Sort the table by QIs
dfsorted = dfin.sort_values(by=LIST_QI)

In [49]:
#Get K number of clusters
K = dfsorted.shape[0]//k #Floor division
K

60

In [50]:
#Random select K distinct records, save to df P:
P = dfsorted.drop_duplicates().sample(n=K)
P = toNode(P)

In [51]:
#Table of records without K selected records
T = dfsorted[~dfsorted.isin(P)].dropna().sort_values(by=LIST_QI)
T = toNode(T)

In [52]:
T_temp = toNode(dfsorted.copy())

In [53]:
dfsorted = toNode(dfsorted)

**Initial clusters** / Khởi tạo K cụm từ bảng P

In [54]:
clusters = list()
counter = 0
for index, row in P.iterrows():
    counter += 1
    clear_output(wait=True)
    clusters.append(Cluster(row))
    print(counter*100/P.shape[0])

100.0


**<font color='red'>Clustering Stage - Giai đoạn phân cụm**

In [55]:
numberOfIteration = T.shape[0]
counter = 0
while not T.empty:
    counter += 1
    #Calculate the distance between each row from T to each cluster's centroid
    distancesList = []
    for cluster in clusters:
        distancesList.append(distance(T.iloc[0], cluster.centroid, dfsorted, qiClass))
   
    #Get the min distance
    minDistance = min(distancesList)
    
    #Add row to its closest cluster and update centroid.
    clusters[distancesList.index(minDistance)].records = clusters[distancesList.index(minDistance)].records.append(T.iloc[0])
    clusters[distancesList.index(minDistance)].updateCentroid()
    
    #Drop row added from T
    T.drop(index=T.iloc[0].name, inplace=True)
    
    clear_output(wait=True)
    print(str(round(counter*100/numberOfIteration, 2)) + "%")

100.0%


In [56]:
for cluster in clusters:
    cluster.records = T
    
counter = 0
numberOfIteration = T_temp.shape[0]

for i in range(len(clusters)):
    counter += 1
    newColString = 'distance_to_cluster_' + str(i)
    T_temp[newColString] = T_temp.apply(lambda x: distance(x, clusters[i].centroid, dfsorted, qiClass), axis=1)
    
    clear_output(wait=True)
    print(str(round(counter*100*k/numberOfIteration, 2)) + "%")

99.46%


In [57]:
#Get random clusters
randomClusterList = []
while len(randomClusterList) < K:
    r = random.randint(0, K-1)
    if r not in randomClusterList:
        randomClusterList.append(r)

In [58]:
numberOfIteration = T_temp.shape[0]
counter = 0
while not T_temp.empty:
    if randomClusterList != []:
        for i in randomClusterList:
            colString = 'distance_to_cluster_' + str(i)
            T_temp = T_temp.sort_values(by=colString, ascending=True)
            clusters[i].records = clusters[i].records.append(T_temp.iloc[0:k])
            #clusters[i].records = clusters[i].records.append(T_temp.iloc[0])
            T_temp = T_temp.iloc[k:]
            if clusters[i].records.shape[0] == k:
                randomClusterList.remove(i)
    else:
        indexCount = 0
        tempIL = 0
        maxIL = 0
        for i in range(len(clusters)):
            clusters[i].updateIL()
            tempIL = clusters[i].informationLoss
            if tempIL > maxIL:
                maxIL = tempIL
                indexCount = i
        clusters[indexCount].records = clusters[indexCount].records.append(T_temp)
        T_temp = T
        if T_temp.empty:
            break

In [59]:
totalIL = 0
for cluster in clusters:
    cluster.updateIL()
    print(cluster.informationLoss)
    totalIL += cluster.informationLoss*cluster.records.shape[0]

5.082191780821918
5.049412915851272
5.435787671232877
4.791095890410959
4.214897260273973
5.791095890410959
5.763698630136986
4.1850627655004535
4.836869361844304
5.353595890410959
3.8368693618443035
5.326198630136986
4.547089041095891
4.301766622118276
3.97556799198129
4.763698630136986
4.791095890410959
4.777397260273973
4.339897260273973
5.086869361844304
4.85056799198129
5.353595890410959
5.326198630136986
5.353595890410959
5.360841964584029
4.836869361844304
5.75
4.836869361844304
5.777397260273973
5.845890410958904
4.589897260273973
5.804794520547945
4.339897260273973
4.791095890410959
4.644691780821918
4.818493150684931
6.012842465753424
5.164383561643835
4.630993150684931
4.339897260273973
4.763698630136986
4.603595890410959
4.339897260273973
5.339897260273973
5.353595890410959
5.777397260273973
5.845890410958904
5.777397260273973
7.06763698630137
4.85056799198129
4.763698630136986
3.2743693618443035
4.969178082191781
4.777397260273973
3.9618693618443035
4.777397260273973
5.449

In [60]:
totalIL

151381.01394324857

In [30]:
for cluster in clusters:
    print(cluster.records.shape[0])

2
2
2
2
2
2
2
2
2
2


#####################**OUTPUT**##########################

In [31]:
#Chuyển dữ liệu số lại như cũ
inv_conversion = {}
for qi in conversion:
    inv_conversion.update({qi: {v: k for k, v in conversion[qi].items()}})
for cluster in clusters:
    for qi in inv_conversion:
        cluster.records[qi] = cluster.records[qi].replace(inv_conversion[qi])

In [32]:
#Remap Value-Label for anonymity
for cluster in clusters:
    for qi in qiClass['numeric']:
        oldValue = cluster.records[qi].unique()
        newValue = []
        for i in range(len(oldValue)):
            if cluster.records[qi].min() == cluster.records[qi].max():
                newValue.append(str(cluster.records[qi].min()))
            else:
                newValue.append('[' + str(cluster.records[qi].min()) + '-' + str(cluster.records[qi].max()) + ']')
        mapDict = dict(zip(oldValue, newValue))
        cluster.records[qi] = cluster.records[qi].replace(mapDict)
    for qi in qiClass['categorical']:
        if NUM_CATEGORICAL:
            oldValue = cluster.records[qi].unique()
            newValue = []
            newValueString = ''
            for i in oldValue:
                newValueString += '|' + str(i) + '|'
            for i in range(len(oldValue)):
                newValue.append(newValueString)
            mapDict = dict(zip(oldValue, newValue))
            cluster.records[qi] = cluster.records[qi].replace(mapDict)
        else:
            oldValue = cluster.records[qi].unique()
            newValue = []
            for i in range(len(oldValue)):
                newValue.append(cluster.centroid[qi])
            mapDict = dict(zip(oldValue, newValue))
            cluster.records[qi] = cluster.records[qi].replace(mapDict)
    for qi in qiClass['hierarchy']:
        oldValue = cluster.records[qi].unique()
        newValue = []
        for i in range(len(oldValue)):
            newValue.append(cluster.centroid[qi].name)
        mapDict = dict(zip(oldValue, newValue))
        cluster.records[qi] = cluster.records[qi].replace(mapDict)

In [33]:
df = T
for cluster in clusters:
    df = pd.concat([df, cluster.records], ignore_index=True)
#df.drop(columns='distance_to_centroid', inplace=True)
for i in range(len(clusters)):
    df.drop(columns='distance_to_cluster_'+str(i), inplace=True)

In [34]:
df[['sex','age','region','married','position','economictype','wage_month']]

,sex,age,region,married,position,economictype,wage_month
0,|female|,[24.0-32.0],region6,1.0,|position3||position2|,|economictype1|,7000
1,|female|,[24.0-32.0],region6,1.0,|position3||position2|,|economictype1|,3000
2,|female|,[30.0-31.0],mienBac,1.0,|position4||position3|,|economictype3||economictype4|,5500
3,|female|,[30.0-31.0],mienBac,1.0,|position4||position3|,|economictype3||economictype4|,1500
4,|female|,[51.0-53.0],Vietnam,1.0,|position2||position1|,|economictype2|,5895
5,|female|,[51.0-53.0],Vietnam,1.0,|position2||position1|,|economictype2|,10400
6,|female|,[28.0-33.0],mienNam,1.0,|position2|,|economictype2|,4000
7,|female|,[28.0-33.0],mienNam,1.0,|position2|,|economictype2|,5000
8,|female|,[21.0-32.0],region2,1.0,|position6||position4|,|economictype3|,2800
9,|female|,[21.0-32.0],region2,1.0,|position6||position4|,|economictype3|,3200


In [37]:
clusters[7].records

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,distance_to_cluster_15,distance_to_cluster_16,distance_to_cluster_17,distance_to_cluster_18,distance_to_cluster_19,distance_to_cluster_20,distance_to_cluster_21,distance_to_cluster_22,distance_to_cluster_23,distance_to_cluster_24
15757,[59.0-79.0],Node('/Any/Worked/With-Pay'),176118.0,Node('/Any/Post-secondary'),14.0,Node('/Any'),Node('/Any/White-collar'),Not-in-family,race,sex,...,6.277561,7.370968,6.159274,7.574194,6.566820,6.564516,7.418280,7.504032,6.491039,7.604839
15875,[59.0-79.0],Node('/Any/Worked/With-Pay'),113959.0,Node('/Any/Post-secondary'),9.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.277561,7.370968,6.159274,7.574194,6.566820,6.564516,7.418280,7.504032,6.491039,7.604839
3056,[59.0-79.0],Node('/Any/Worked/With-Pay'),361494.0,Node('/Any/Post-secondary'),9.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.277561,7.370968,6.159274,7.574194,6.566820,6.564516,7.418280,7.504032,6.491039,7.604839
582,[59.0-79.0],Node('/Any/Worked/With-Pay'),199713.0,Node('/Any/Post-secondary'),9.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.277561,7.370968,6.159274,7.574194,6.566820,6.564516,7.418280,7.504032,6.491039,7.604839
27645,[59.0-79.0],Node('/Any/Worked/With-Pay'),170104.0,Node('/Any/Post-secondary'),13.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.277561,7.370968,6.159274,7.574194,6.566820,6.564516,7.418280,7.504032,6.491039,7.604839
5129,[59.0-79.0],Node('/Any/Worked/With-Pay'),143932.0,Node('/Any/Post-secondary'),14.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.293690,7.387097,6.175403,7.590323,6.582949,6.580645,7.434409,7.520161,6.507168,7.620968
28259,[59.0-79.0],Node('/Any/Worked/With-Pay'),95445.0,Node('/Any/Post-secondary'),10.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.293690,7.387097,6.175403,7.590323,6.582949,6.580645,7.434409,7.520161,6.507168,7.620968
12857,[59.0-79.0],Node('/Any/Worked/With-Pay'),52900.0,Node('/Any/Post-secondary'),4.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.293690,7.387097,6.175403,7.590323,6.582949,6.580645,7.434409,7.520161,6.507168,7.620968
6117,[59.0-79.0],Node('/Any/Worked/With-Pay'),187124.0,Node('/Any/Post-secondary'),9.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.293690,7.387097,6.175403,7.590323,6.582949,6.580645,7.434409,7.520161,6.507168,7.620968
2956,[59.0-79.0],Node('/Any/Worked/With-Pay'),232618.0,Node('/Any/Post-secondary'),13.0,Node('/Any'),Node('/Any/White-collar'),Husband,race,sex,...,6.293690,7.387097,6.175403,7.590323,6.582949,6.580645,7.434409,7.520161,6.507168,7.620968


In [34]:
import pandas as pd
import math
import numpy as np
import re
import random
from anytree import Node, RenderTree, PreOrderIter
from anytree.util import commonancestors

qiClass = {}
#Hàm phân loại quasi-identifiers theo numeric or categorical or hierarchy
def classifyNumCtg(df, qi):
    #Convert numeric to it's type
    for i in qi:
        df[i] = pd.to_numeric(df[i], errors='ignore')
    #Classify QI Numeric and QI Categorical
    qiNumeric = []
    qiCategorical = []
    for i in qi:
        if df[i].dtype == 'O':
            qiCategorical.append(i)
        else:
            qiNumeric.append(i)
    return {'numeric':qiNumeric, 'categorical':qiCategorical, 'hierarchy':{}}

#Hàm chuyển đổi input từ user sang dạng cây phân cấp:
#TREE STRING EXAMPLE: A(B/C) với A là root B, C là node con của A
#-------------------- A(B(D/E)/C(G/H)) A là root; D, E là node con của B
#-------------------- G, H là node con của C; B, C là node con của A 
def toTree(tree_string):
    x = re.split('[()]', tree_string)
    x = [i for i in x if i]
    rootNode = 0
    parentNode = []
    leafNode = []
    counterParent = 0
    counterLeaf = 0
    for item in x:
        if len(re.findall('/', item)) == 0:
            flag = 0
            if rootNode == 0:
                rootNode = Node(item)
            else:
                if counterParent == 0:
                    parentNode.append(Node(item, parent=rootNode))
                else:
                    parentNode.append(Node(item, parent=parentNode[counterParent-1]))
                counterParent = counterParent + 1
        else:
            if not re.search('^/', item):
                if flag != 1:
                    for i in re.split('/', item):
                        try:
                            leafNode.append(Node(i, parent=parentNode[counterParent-1]))
                        except:
                            leafNode.append(Node(i, parent=rootNode))
                        counterLeaf = counterLeaf + 1
                if flag == 1:
                    counterLeaf = counterLeaf - 1
                    parentNode.append(leafNode[counterLeaf])
                    leafNode.remove(leafNode[counterLeaf])
                    counterParent = counterParent + 1
                    for i in re.split('/', item):
                        leafNode.append(Node(i, parent=parentNode[counterParent-1]))
                        counterLeaf = counterLeaf + 1
                flag = 1
            else:
                if len(re.findall('/', item)) < 2:
                    if flag != 2:
                        if item == x[-1]:
                            if str('))' + item) in tree_string:
                                leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent))
                            else:
                                leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent))
                            counterLeaf = counterLeaf + 1
                        else:
                            if str('))))' + item) in tree_string:
                                parentNode.append(Node(item.split('/')[1], parent=rootNode))
                            elif str('))' + item) in tree_string:
                                parentNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent))
                            else:
                                parentNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent))
                            counterParent = counterParent + 1
                    if flag == 2:
                        counterParent = counterParent - 1
                        leafNode.append(parentNode[counterParent])
                        parentNode.remove(parentNode[counterParent])
                        counterLeaf = counterLeaf + 1
                        if str(')' + item) in tree_string:
                            leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent.parent))
                        else:
                            leafNode.append(Node(item.split('/')[1], parent=parentNode[counterParent-1].parent.parent))
                        counterLeaf = counterLeaf + 1
                    flag = 2
                else:
                    pass
    return [rootNode, parentNode, leafNode]

#Hàm chuyển QI kiểu categorical về dạng numeric:
def convertQIValue(df, qilist):
    fullMapDict = {}
    for qi in qilist:
        oldValue = df.sort_values(by=qi)[qi].unique()
        newValue = []
        for i in range(len(df[qi].unique())):
            if len(df[qi].unique()) == 2:
                #newValue.append(qi + str(i))
                newValue.append(i)
            else:
                newValue.append(i+1)
        mapDict = dict(zip(oldValue, newValue))
        df[qi] = df[qi].replace(mapDict)
        fullMapDict.update({qi: mapDict})
    return fullMapDict

## Hàm tính khoảng cách giữa 2 records (rows)
NUM_CATEGORICAL = True
def distance(r1, r2, df, qiClass): #(row1, row2, dataframe, quasi-identifers class)
    totalDistance = 0
    numDistance = 0
    catgDistance = 0
    hieDistance = 0
    for qi in qiClass['numeric']:
        if df[qi].max() == df[qi].min():
            numDistance += 0
        else:
            numDistance += abs(r1[qi]-r2[qi])/(df[qi].max() - df[qi].min())
    for qi in qiClass['categorical']:
        if NUM_CATEGORICAL:
            if df[qi].max() == df[qi].min():
                catgDistance += 0
            else:
                catgDistance += abs(r1[qi]-r2[qi])/(df[qi].max() - df[qi].min())
        else:
            if r1[qi] != r2[qi]:
                catgDistance += 1
    for qi in qiClass['hierarchy'].keys():
        if r1[qi] != r2[qi]:
            r1_ac = set(r1[qi].ancestors)
            r2_ac = set(r2[qi].ancestors)
            try:
                comanc = list(r1_ac.intersection(r2_ac))[-1]
            except:
                comanc = qiClass['hierarchy'][qi]['root']
            numLeavesCA = len(list(PreOrderIter(comanc, filter_=lambda node: node.is_leaf)))
            totalLeaves = len(qiClass['hierarchy'][qi]['children'])
            hieDistance += numLeavesCA/totalLeaves
    totalDistance = numDistance + catgDistance + hieDistance
    return totalDistance

#Convert string type to tree node type:
#Hàm chuyển giá trị kiểu chuỗi sang kiểu node cây
def toNode(df, qiClass):
    for qi in qiClass['hierarchy']:
        leavesDict = {}
        for leaf in qiClass['hierarchy'][qi]['children']:
            leavesDict.update({leaf.name:leaf})
        df[qi] = df[qi].replace(leavesDict)
    return df

## Định nghĩa lớp phân cụm:
dfsorted = pd.DataFrame()
class Cluster:
    def __init__(self, centroid):
        self.centroid = centroid
        self.records = pd.DataFrame(columns=self.centroid.index)
        self.records = self.records.append(self.centroid)
        self.informationLoss = 0
    def updateCentroid(self, qiClass):
        for qi in qiClass['numeric']:
            self.centroid[qi] = self.records[qi].mean()
        for qi in qiClass['categorical']:
            if NUM_CATEGORICAL:
                self.centroid[qi] = self.records[qi].mean()
            else:
                if len(self.records[qi].unique()) > 1:
                    self.centroid[qi] = qi
                else:
                    self.centroid[qi] = self.records[qi].unique()[0]
        for qi in qiClass['hierarchy']:
            listAncestor = []
            for item in self.records[qi].unique():
                listAncestor.append(set(item.ancestors))
            for n in range(len(listAncestor)):
                commonA = listAncestor[0].intersection(listAncestor[n])
            commonA = list(commonA)
            lowestCommonA = commonA[0]
            for i in commonA:
                if len(i.path) > len(lowestCommonA.path):
                    lowestCommonA = i
            self.centroid[qi] = lowestCommonA
    def updateIL(self, qiClass, dfsorted):
        PIL = 0
        numDistance = 0
        catgDistance = 0
        hieDistance = 0
        for qi in qiClass['numeric']:
            if dfsorted[qi].max() == dfsorted[qi].min():
                numDistance += 0
            else:
                numDistance += abs(self.records[qi].max() - self.records[qi].min())/(dfsorted[qi].max() - dfsorted[qi].min())
        for qi in qiClass['categorical']:
            if NUM_CATEGORICAL:
                if dfsorted[qi].max() == dfsorted[qi].min():
                    catgDistance += 0
                else:
                    catgDistance += abs(self.records[qi].max() - self.records[qi].min())/(self[qi].max() - self[qi].min())
            else:
                if len(self.records[qi].unique()) > 1:
                    catgDistance += 1
        for qi in qiClass['hierarchy'].keys():
            listAncestor = []
            for item in self.records[qi].unique():
                listAncestor.append(set(item.ancestors))
            for n in range(len(listAncestor)):
                commonA = listAncestor[0].intersection(listAncestor[n])
            lowestCommonA = list(commonA)[-1]

            numLeavesCA = len(list(PreOrderIter(lowestCommonA, filter_=lambda node: node.is_leaf)))
            totalLeaves = len(qiClass['hierarchy'][qi]['children'])
            hieDistance += numLeavesCA/totalLeaves
        PIL = numDistance + catgDistance + hieDistance
        self.informationLoss = PIL

## Hàm chạy giải thuật ##
def anonymizeDataFrame(dfin, LIST_QI, NUM_CATEGORICAL=True, k=2):
    #Phân loại Quasi-identifiers
    qiClass = classifyNumCtg(dfin, LIST_QI)

    #Chuyển các categorical QIs về dạng số nếu NUM_CATEGORICAL = True
    if NUM_CATEGORICAL:
        conversion = convertQIValue(dfin, qiClass['categorical'])
    else:
        conversion = {}

    #Sort the table by QIs
    dfsorted = dfin.sort_values(by=LIST_QI)

    #Get K number of clusters
    K = dfsorted.shape[0]//int(k) #Chia lấy phần nguyên dưới
    
    #Random select K distinct records, save to df P:
    P = dfsorted.drop_duplicates().sample(n=K)
    P = toNode(P, qiClass)

    #Table of records without K selected records
    T = dfsorted[~dfsorted.isin(P)].dropna().sort_values(by=LIST_QI)
    T = toNode(T, qiClass)
    T_temp = toNode(dfsorted.copy(), qiClass)
    dfsorted = toNode(dfsorted, qiClass)

    #Khởi tạo cụm
    clusters = list()
    for index, row in P.iterrows():
        clusters.append(Cluster(row))

    #Phân cụm
    while not T.empty:
        #Calculate the distance between each row from T to each cluster's centroid
        distancesList = []
        for cluster in clusters:
            distancesList.append(distance(T.iloc[0], cluster.centroid, dfsorted, qiClass))
    
        #Get the min distance
        minDistance = min(distancesList)
        
        #Add row to its closest cluster and update centroid.
        clusters[distancesList.index(minDistance)].records = clusters[distancesList.index(minDistance)].records.append(T.iloc[0])
        clusters[distancesList.index(minDistance)].updateCentroid(qiClass)
        
        #Drop row added from T
        T.drop(index=T.iloc[0].name, inplace=True)
        
    #Điều chỉnh cụm
    for cluster in clusters:
        cluster.records = T
    for i in range(len(clusters)):
        newColString = 'distance_to_cluster_' + str(i)
        T_temp[newColString] = T_temp.apply(lambda x: distance(x, clusters[i].centroid, dfsorted, qiClass), axis=1)
    
    #Get random clusters
    randomClusterList = []
    while len(randomClusterList) < K:
        r = random.randint(0, K-1)
        if r not in randomClusterList:
            randomClusterList.append(r)
    
    while not T_temp.empty:
        if randomClusterList != []:
            for i in randomClusterList:
                colString = 'distance_to_cluster_' + str(i)
                T_temp = T_temp.sort_values(by=colString, ascending=True)
                clusters[i].records = clusters[i].records.append(T_temp.iloc[0:k])
                T_temp = T_temp.iloc[k:]
                if clusters[i].records.shape[0] == k:
                    randomClusterList.remove(i)
        else:
            indexCount = 0
            tempIL = 0
            maxIL = 0
            for i in range(len(clusters)):
                clusters[i].updateIL(qiClass, dfsorted)
                tempIL = clusters[i].informationLoss
                if tempIL > maxIL:
                    maxIL = tempIL
                    indexCount = i
            clusters[indexCount].records = clusters[indexCount].records.append(T_temp)
            T_temp = T
            if T_temp.empty:
                break
    
    #Chuyển dữ liệu số lại như cũ
    inv_conversion = {}
    for qi in conversion:
        inv_conversion.update({qi: {v: k for k, v in conversion[qi].items()}})
    for cluster in clusters:
        for qi in inv_conversion:
            cluster.records[qi] = cluster.records[qi].replace(inv_conversion[qi])

    #Remap Value-Label for anonymity
    for cluster in clusters:
        for qi in qiClass['numeric']:
            oldValue = cluster.records[qi].unique()
            newValue = []
            for i in range(len(oldValue)):
                if cluster.records[qi].min() == cluster.records[qi].max():
                    newValue.append(str(cluster.records[qi].min()))
                else:
                    newValue.append('[' + str(cluster.records[qi].min()) + '-' + str(cluster.records[qi].max()) + ']')
            mapDict = dict(zip(oldValue, newValue))
            cluster.records[qi] = cluster.records[qi].replace(mapDict)
        for qi in qiClass['categorical']:
            if NUM_CATEGORICAL:
                oldValue = cluster.records[qi].unique()
                newValue = []
                newValueString = ''
                for i in oldValue:
                    newValueString += '|' + str(i) + '|'
                for i in range(len(oldValue)):
                    newValue.append(newValueString)
                mapDict = dict(zip(oldValue, newValue))
                cluster.records[qi] = cluster.records[qi].replace(mapDict)
            else:
                oldValue = cluster.records[qi].unique()
                newValue = []
                for i in range(len(oldValue)):
                    newValue.append(cluster.centroid[qi])
                mapDict = dict(zip(oldValue, newValue))
                cluster.records[qi] = cluster.records[qi].replace(mapDict)
        for qi in qiClass['hierarchy']:
            oldValue = cluster.records[qi].unique()
            newValue = []
            for i in range(len(oldValue)):
                newValue.append(cluster.centroid[qi].name)
            mapDict = dict(zip(oldValue, newValue))
            cluster.records[qi] = cluster.records[qi].replace(mapDict)
    
    dfout = T
    for cluster in clusters:
        dfout = pd.concat([dfout, cluster.records], ignore_index=True)
    for i in range(len(clusters)):
        dfout.drop(columns='distance_to_cluster_'+str(i), inplace=True)

    return dfout

dfin = pd.read_csv('input.csv', index_col=0)
dfin = dfin.sample(20)
k = 2
LIST_QI = ['sex','age','region','married','position','economictype']
LIST_QI = ['region', 'sex', 'age', 'married', 'position', 'economictype']
NUM_CATEGORICAL = True
dfout = anonymizeDataFrame(dfin, LIST_QI, NUM_CATEGORICAL, k)

In [35]:
dfout

,wage_month,region,head,sex,age,married,schooling,position,economictype,ky hop dong lao dong?,...,urban,Kinh,household_size,poor,houses,pay_rent,other_land/house,wealthindex,t_quantity durables,t_value durables
0,5200,|region4||region5|,0.0,|female|,[23.0-26.0],[0.0-1.0],14.0,|position4|,|economictype3|,1.0,...,1.0,1.0,6.0,0.0,1.0,0.0,0.0,125.0,19.0,44870.0
1,5500,|region4||region5|,0.0,|female|,[23.0-26.0],[0.0-1.0],9.0,|position4|,|economictype3|,1.0,...,0.0,1.0,5.0,0.0,1.0,0.0,0.0,130.0,33.0,164400.0
2,1200,|region5|,0.0,|female|,[37.0-46.0],1.0,4.0,|position6||position3|,|economictype4||economictype2|,2.0,...,0.0,1.0,4.0,0.0,1.0,0.0,0.0,75.0,5.0,4500.0
3,6000,|region5|,0.0,|female|,[37.0-46.0],1.0,12.0,|position6||position3|,|economictype4||economictype2|,1.0,...,1.0,1.0,3.0,0.0,1.0,0.0,0.0,160.0,15.0,36800.0
4,2340,|region5||region6|,1.0,|male|,[47.0-50.0],1.0,12.0,|position6||position4|,|economictype4|,2.0,...,0.0,1.0,3.0,0.0,1.0,0.0,0.0,135.0,12.0,11060.0
5,3750,|region5||region6|,1.0,|male|,[47.0-50.0],1.0,9.0,|position6||position4|,|economictype4|,2.0,...,0.0,1.0,3.0,0.0,1.0,0.0,0.0,120.0,14.0,41430.0
6,1000,|region5||region2|,0.0,|male||female|,[26.0-50.0],[0.0-1.0],6.0,|position4||position2|,|economictype4||economictype2|,2.0,...,0.0,1.0,5.0,0.0,1.0,0.0,0.0,135.0,24.0,45800.0
7,5950,|region5||region2|,0.0,|male||female|,[26.0-50.0],[0.0-1.0],14.0,|position4||position2|,|economictype4||economictype2|,1.0,...,0.0,0.0,6.0,0.0,1.0,0.0,0.0,100.0,16.0,21200.0
8,5500,|region1||region5|,1.0,|male||female|,[41.0-54.0],1.0,9.0,|position4||position6|,|economictype4|,2.0,...,0.0,1.0,5.0,0.0,1.0,0.0,0.0,155.0,21.0,15295.0
9,500,|region1||region5|,0.0,|male||female|,[41.0-54.0],1.0,0.0,|position4||position6|,|economictype4|,2.0,...,0.0,1.0,5.0,1.0,1.0,0.0,0.0,115.0,2.0,700.0
